In [7]:
%glue_version 2.0
%number_of_workers 2
%worker_type G.2X
%idle_timeout 60
%region us-east-2
%iam_role arn:aws:iam::0000000000:role/AWSGlueServiceRoleJupyter

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Setting Glue version to: 2.0
Previous number of workers: 5
Setting new number of workers to: 2
Previous worker type: G.1X
Setting new worker type to: G.2X
Current idle_timeout is 2880 minutes.
idle_timeout has been set to 60 minutes.
Previous region: None
Setting new region to: us-east-2
Region is set to: us-east-2
Current iam_role is None
iam_role has been set to arn:aws:iam::0000000000:role/AWSGlueServiceRoleJupyter.


In [1]:
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Authenticating with profile=default
glue_role_arn defined by user: arn:aws:iam::0000000000:role/AWSGlueServiceRoleJupyter
Trying to create a Glue session for the kernel.
Worker Type: G.2X
Number of Workers: 2
Session ID: a292dec3-82a6-4f45-b150-c19ef5aeaa23
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.2
--enable-glue-datacatalog true
Waiting for session a292dec3-82a6-4f45-b150-c19ef5aeaa23 to get into ready status...
Session a292dec3-82a6-4f45-b150-c19ef5aeaa23 has been created.



In [2]:
%%sql 
show tables in `covid-19`

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|covid-19|alleninstitute_co...|      false|
|covid-19|alleninstitute_me...|      false|
|covid-19|aspirevc_crowd_tr...|      false|
|covid-19|aspirevc_crowd_tr...|      false|
|covid-19|cdc_moderna_vacci...|      false|
|covid-19|cdc_pfizer_vaccin...|      false|
|covid-19|       country_codes|      false|
|covid-19|  county_populations|      false|
|covid-19|covid_knowledge_g...|      false|
|covid-19|covid_knowledge_g...|      false|
|covid-19|covid_knowledge_g...|      false|
|covid-19|covid_knowledge_g...|      false|
|covid-19|covid_knowledge_g...|      false|
|covid-19|covid_knowledge_g...|      false|
|covid-19|covid_testing_sta...|      false|
|covid-19|covid_testing_us_...|      false|
|covid-19|covid_testing_us_...|      false|
|covid-19|      covidcast_data|      false|
|covid-19|  covidcast_metadata|      false|
|covid-19|enigma_aggregatio...| 

In [3]:
%%sql 
select * from `covid-19`.county_populations sort by `population estimate 2018` desc limit 10

+--------------+-----+---------------+-----------+------------------------+
|            id|  id2|         county|      state|population estimate 2018|
+--------------+-----+---------------+-----------+------------------------+
|            Id|  Id2|         County|      State|    Population Estima...|
|0500000US01085| 1085|        Lowndes|    Alabama|                    9974|
|0500000US06057| 6057|         Nevada| California|                   99696|
|0500000US29189|29189|      St. Louis|   Missouri|                  996945|
|0500000US22021|22021|Caldwell Parish|  Louisiana|                    9960|
|0500000US06019| 6019|         Fresno| California|                  994400|
|0500000US28143|28143|         Tunica|Mississippi|                    9944|
|0500000US05051| 5051|        Garland|   Arkansas|                   99154|
|0500000US29079|29079|         Grundy|   Missouri|                    9914|
|0500000US27063|27063|        Jackson|  Minnesota|                    9911|
+-----------

In [4]:
dyf = glueContext.create_dynamic_frame.from_catalog(
    database = "covid-19", table_name = "county_populations")
dyf.printSchema()

root
|-- id: string
|-- id2: string
|-- county: string
|-- state: string
|-- population estimate 2018: string


In [5]:
mapped = dyf.apply_mapping(
    mappings = [
        ("id", "string", "id", "string"),
        ("id2", "string", "simple_id", "int"),
        ("county", "string", "county", "string"),
        ("state", "string", "state", "string"),
        ("population estimate 2018", "string", "pop_est_2018", "long")
    ]
)

mapped.printSchema()
mapped_df = mapped.toDF()
mapped_df.show()

root
|-- id: string
|-- simple_id: int
|-- county: string
|-- state: string
|-- pop_est_2018: long

+--------------+---------+---------+-------+------------+
|            id|simple_id|   county|  state|pop_est_2018|
+--------------+---------+---------+-------+------------+
|0500000US01001|     1001|  Autauga|Alabama|       55601|
|0500000US01003|     1003|  Baldwin|Alabama|      218022|
|0500000US01005|     1005|  Barbour|Alabama|       24881|
|0500000US01007|     1007|     Bibb|Alabama|       22400|
|0500000US01009|     1009|   Blount|Alabama|       57840|
|0500000US01011|     1011|  Bullock|Alabama|       10138|
|0500000US01013|     1013|   Butler|Alabama|       19680|
|0500000US01015|     1015|  Calhoun|Alabama|      114277|
|0500000US01017|     1017| Chambers|Alabama|       33615|
|0500000US01019|     1019| Cherokee|Alabama|       26032|
|0500000US01021|     1021|  Chilton|Alabama|       44153|
|0500000US01023|     1023|  Choctaw|Alabama|       12841|
|0500000US01025|     1025|   C

In [6]:
spark.sql("create database if not exists demo")

DataFrame[]


In [7]:
%%sql
show databases

+------------+
|databaseName|
+------------+
|    covid-19|
|     default|
|        demo|
|     test-db|
+------------+


In [8]:
s3_bucket = "glue-sink-20230404"
sink = glueContext.getSink(
    path = f"s3://{s3_bucket}/interactive-session/pops/",
    connection_type = "s3",
    updateBehavior = "UPDATE_IN_DATABASE",
    paritionKyes = [],
    compress = "snappy",
    enableUpdateCatalog = True,
    transformation_ctx = "s3output"
)

sink.setCatalogInfo(catalogDatabase="demo", catalogTableName="pops")
sink.setFormat("glueparquet")
sink.writeFrame(mapped)

In [9]:
%%sql
select * from demo.pops


+--------------+---------+---------+-------+------------+
|            id|simple_id|   county|  state|pop_est_2018|
+--------------+---------+---------+-------+------------+
|0500000US01001|     1001|  Autauga|Alabama|       55601|
|0500000US01003|     1003|  Baldwin|Alabama|      218022|
|0500000US01005|     1005|  Barbour|Alabama|       24881|
|0500000US01007|     1007|     Bibb|Alabama|       22400|
|0500000US01009|     1009|   Blount|Alabama|       57840|
|0500000US01011|     1011|  Bullock|Alabama|       10138|
|0500000US01013|     1013|   Butler|Alabama|       19680|
|0500000US01015|     1015|  Calhoun|Alabama|      114277|
|0500000US01017|     1017| Chambers|Alabama|       33615|
|0500000US01019|     1019| Cherokee|Alabama|       26032|
|0500000US01021|     1021|  Chilton|Alabama|       44153|
|0500000US01023|     1023|  Choctaw|Alabama|       12841|
|0500000US01025|     1025|   Clarke|Alabama|       23920|
|0500000US01027|     1027|     Clay|Alabama|       13275|
|0500000US0102

In [10]:
%%sql
select count(1) from demo.pops

+--------+
|count(1)|
+--------+
|   12880|
+--------+


In [9]:
%pwd

'/Users/soonbeom/glue'